<a href="https://colab.research.google.com/github/sauravsolanki/Kannada_Mnist/blob/main/Kannada_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root= "/content/drive/My Drive/colab_data/data/kannada_mnist/"

In [ ]:
from __future__ import print_function, division


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

# define model
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy


In [ ]:
class KannadaMnistDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None,phase = "train"):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = pd.read_csv(root + csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        t = self.data.iloc[idx]
        label = int(t[0])
        img = t[1:]/255.0
        image = np.array(img).reshape(1,28,28)
        
        if self.phase =="train":
            sample = {'image': image, 'label': label}
            if self.transform:
                sample = self.transform(sample)
            
        else:
            sample = {'image': image, 'id': label}
            if self.transform:
                sample = self.transform(sample)
        return sample

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(1014, 512)  # 6*6 from image dimension
        self.drop_layer = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(512, 10)
        # self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = self.drop_layer(F.relu(self.fc1(x)))
        # x = self.drop_layer(F.relu(self.fc2(x)))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1014, out_features=512, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
def train_model(model,dataloader,dataset_sizes,criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for sample in dataloader[phase]:
                inputs, labels  =sample["image"],sample["label"]
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(type(input[0])
                # zero the parameter gradients
                
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #if phase == 'train':
            #    scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('Epoch:{} {} Loss: {:.4f} Acc: {:.4f}'.format(epoch,phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                #save the model
                torch.save({'epoch': num_epochs,
                    'model_state_dict': best_model_wts,
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': epoch_loss,
                    }, PATH)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
batch_size=32

In [ ]:
train_dataset = KannadaMnistDataset(csv_file='train.csv',root_dir='',transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=256)

In [ ]:
val_dataset = KannadaMnistDataset(csv_file='Dig-MNIST.csv',root_dir='.',transform=None)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,shuffle=False, num_workers=256)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PATH="./best_model1.pth"
dataset_sizes={"train":len(train_dataset),"val":len(val_dataset)}
dataloader={"train":train_dataloader,"val":val_dataloader}

model = Net()
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model,dataloader,dataset_sizes,criterion, optimizer, scheduler=None, num_epochs=100)

Epoch:0 train Loss: 0.0739 Acc: 0.9777
Epoch:0 val Loss: 1.8441 Acc: 0.6747

Epoch:1 train Loss: 0.0548 Acc: 0.9833
Epoch:1 val Loss: 1.5548 Acc: 0.7174

Epoch:2 train Loss: 0.0417 Acc: 0.9868
Epoch:2 val Loss: 2.1069 Acc: 0.6877

Epoch:3 train Loss: 0.0359 Acc: 0.9888
Epoch:3 val Loss: 1.7755 Acc: 0.7371

Epoch:4 train Loss: 0.0289 Acc: 0.9911
Epoch:4 val Loss: 1.8567 Acc: 0.7341

Epoch:5 train Loss: 0.0235 Acc: 0.9925
Epoch:5 val Loss: 2.2506 Acc: 0.7203

Epoch:6 train Loss: 0.0212 Acc: 0.9928
Epoch:6 val Loss: 1.9103 Acc: 0.7334

Epoch:7 train Loss: 0.0178 Acc: 0.9939
Epoch:7 val Loss: 2.2230 Acc: 0.7224

Epoch:8 train Loss: 0.0166 Acc: 0.9944
Epoch:8 val Loss: 2.0427 Acc: 0.7407

Epoch:9 train Loss: 0.0151 Acc: 0.9950
Epoch:9 val Loss: 2.4515 Acc: 0.7360

Epoch:10 train Loss: 0.0123 Acc: 0.9958
Epoch:10 val Loss: 2.3257 Acc: 0.7576

Epoch:11 train Loss: 0.0116 Acc: 0.9962
Epoch:11 val Loss: 2.4876 Acc: 0.7333

Epoch:12 train Loss: 0.0110 Acc: 0.9961
Epoch:12 val Loss: 2.9589 Acc: 0

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1014, out_features=512, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [ ]:
# class wise accuracy
# load best model 
model.load_state_dict(torch.load(PATH)["model_state_dict"])
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for sample in val_dataloader:
        inputs, labels = sample["image"],sample["label"]

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs.float())
        #print(labels.item)
        _, predicted = torch.max(outputs, 1)
        #print(predicted)
        c = (predicted == labels).squeeze()
        #print(c)
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
        


for i in range(10):
    print('Accuracy of %d : %2d %%' % (
        i, 100 * class_correct[i] / class_total[i]))

In [ ]:
def test_model(dataloader,model):
    # load best model weights
    #model.load_state_dict(best_model_wts)
    since = time.time()
    p=""
    model.eval()
    with torch.no_grad():
        for sample in dataloader:
                inputs, idx  =sample["image"],sample["id"]                
                inputs = inputs.to(device)
                
                outputs = model(inputs.float())
                _, preds = torch.max(outputs, 1)
                #idx,preds
                #print(idx,preds)
                for i,j in zip(idx,preds):
                    p+=str(i.item())+","+str(j.item())+"\n"
                     
    time_elapsed = time.time() - since
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return p

In [ ]:
test_dataset = KannadaMnistDataset(csv_file='test.csv',
                                           root_dir='',
                                           transform=None,
                                           phase="test")
test_dataloader = DataLoader(test_dataset, batch_size=4,shuffle=False, num_workers=0)

submit=test_model(test_dataloader,model)

In [ ]:
submitt = "id,label\n"+submit
with open("submission.csv","w") as f:
    f.write(submitt)

In [ ]:
from google.colab import files
files.download('submission.csv') 